
## Google analytics customer revenue predictions
### By Ahmed Eweed  

### You can find me on github at [github.com/AhmedEweed](https://github.com/AhmedEweed)

In [3]:
#import needed libraries
import pandas as pd

#read data 
# I solved the mixed datatype problem in column 3 'fullVisitorId' by defining it to be string 
# that should stop the warning message.
df = pd.read_csv("../train.csv", dtype = {'fullVisitorId': 'str'})

In [76]:
#look at dataset dimensions
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
channelGrouping         903653 non-null object
date                    903653 non-null int64
device                  903653 non-null object
fullVisitorId           903653 non-null object
geoNetwork              903653 non-null object
sessionId               903653 non-null object
socialEngagementType    903653 non-null object
totals                  903653 non-null object
trafficSource           903653 non-null object
visitId                 903653 non-null int64
visitNumber             903653 non-null int64
visitStartTime          903653 non-null int64
dtypes: int64(4), object(8)
memory usage: 55.2+ MB


#####    the dataset has nearly 904k rows and 12 columns
#####    8 of those columns are strings and 4 are numbers or so says the dataset
#####    we have 4 columns that contain json data ('device', 'geoNetwork', 'totals', 'trafficSource' )
#####    let's have a deeper look at each column's data!

In [78]:
# look at the data
df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [100]:
# look at 'channelGrouping' values
df['channelGrouping'].value_counts()

Organic Search    381561
Social            226117
Direct            143026
Referral          104838
Paid Search        25326
Affiliates         16403
Display             6262
(Other)              120
Name: channelGrouping, dtype: int64

##### this is the reach source by channel

In [80]:
# look at 'date' column
# date is in Y/M/D format Remember!
df['date'].head()

0    20160902
1    20160902
2    20160902
3    20160902
4    20160902
Name: date, dtype: int64

##### this is definitely wrong! as date is represented by integers.
##### we will fix that.

In [98]:
# look at 'device' column contents
df['device'][1]

'{"browser": "Firefox", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Macintosh", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [82]:
# look at 'fullVisitorId' contents
df['fullVisitorId'].head()

0    1131660440785968503
1     377306020877927890
2    3895546263509774583
3    4763447161404445595
4      27294437909732085
Name: fullVisitorId, dtype: object

In [83]:
# look at 'geoNetwork' column contents
df['geoNetwork'][2]

'{"continent": "Europe", "subContinent": "Southern Europe", "country": "Spain", "region": "Community of Madrid", "metro": "(not set)", "city": "Madrid", "cityId": "not available in demo dataset", "networkDomain": "unknown.unknown", "latitude": "not available in demo dataset", "longitude": "not available in demo dataset", "networkLocation": "not available in demo dataset"}'

In [84]:
# look at 'sessionId' content
df['sessionId'].head()

0    1131660440785968503_1472830385
1     377306020877927890_1472880147
2    3895546263509774583_1472865386
3    4763447161404445595_1472881213
4      27294437909732085_1472822600
Name: sessionId, dtype: object

In [85]:
# look at 'socialEngagmentType' value counts
df['socialEngagementType'].value_counts()

Not Socially Engaged    903653
Name: socialEngagementType, dtype: int64

##### looks like all values at this column are the same as 'Not Socially Engaged'

In [99]:
# look at 'totals' contents
df['totals'][1]

'{"visits": "1", "hits": "1", "pageviews": "1", "bounces": "1", "newVisits": "1"}'

In [87]:
# look at 'trafficSource' content
df['trafficSource'][4]

'{"campaign": "(not set)", "source": "google", "medium": "organic", "keyword": "(not provided)", "adwordsClickInfo": {"criteriaParameters": "not available in demo dataset"}, "isTrueDirect": true}'

In [88]:
# look at 'visitId' contents
df['visitId'].head()

0    1472830385
1    1472880147
2    1472865386
3    1472881213
4    1472822600
Name: visitId, dtype: int64

In [89]:
# look at 'visitNumber' contents
df['visitNumber'].head()

0    1
1    1
2    1
3    1
4    2
Name: visitNumber, dtype: int64

In [90]:
# look at 'visitStartTime' contents
df['visitStartTime'].head()

0    1472830385
1    1472880147
2    1472865386
3    1472881213
4    1472822600
Name: visitStartTime, dtype: int64

#### Now we will fix the date problem


In [91]:
# first we will convert the integer to string
df['date'] = df['date'].astype(str)

In [92]:
# make sure the data type is converted 
df['date'].head()

0    20160902
1    20160902
2    20160902
3    20160902
4    20160902
Name: date, dtype: object

In [93]:
# now convert the str to datetime on the format Y-M-D
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')

In [94]:
# make sure the conversion is done 
df['date'].head()

0   2016-09-02
1   2016-09-02
2   2016-09-02
3   2016-09-02
4   2016-09-02
Name: date, dtype: datetime64[ns]